In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import re
import json

In [2]:
_bb=pd.read_csv('bkk_budget_ocr.csv',index_col=0)
_bb=_bb[~_bb['pagenum'].isna()]

/Users/napatswift/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# number of document
len(_bb.pdf.unique())

385

In [4]:
_bb['pagenum'] = _bb['pagenum'].astype(int)
_bb['text'] = _bb['text'].astype(str)

select test set

remove unwanted document (just for develop)

In [5]:
_bb

,pdf,pagenum,image_path,x0,y0,x1,y1,text,ocr-text
0,pdf/61/A20171003165836.pdf,0,A20171003165836/00.png,886.109985,218.514427,944.322144,260.634430,161,NaN
1,pdf/61/A20171003165836.pdf,0,A20171003165836/00.png,784.949982,352.531105,1045.670013,394.651108,ส/านักงานเขตบางพลัด,NaN
2,pdf/61/A20171003165836.pdf,0,A20171003165836/00.png,436.019989,474.211075,666.247696,516.331100,งบประมาณรายจ่าย,NaN
3,pdf/61/A20171003165836.pdf,0,A20171003165836/00.png,436.019989,541.621170,455.013062,583.741196,ปี,NaN
4,pdf/61/A20171003165836.pdf,0,A20171003165836/00.png,464.460022,541.621170,538.070297,583.741196,2560,NaN
...,...,...,...,...,...,...,...,...,...
1693958,pdf/65/65071.pdf,61,65071/61.png,352.500000,1624.007996,452.317520,1670.340088,07199-3,NaN
1693959,pdf/65/65071.pdf,61,65071/61.png,512.339996,1624.007996,543.382828,1670.340088,(9),NaN
1693960,pdf/65/65071.pdf,61,65071/61.png,552.300018,1624.007996,900.000641,1670.340088,ค่ำใช้จ่ำยในกำรสอนภำษำมลำยู,NaN
1693961,pdf/65/65071.pdf,61,65071/61.png,1497.899963,1626.527893,1595.576111,1672.859985,"144,000",NaN


In [9]:
_bb=_bb.assign(fisical_year=_bb.pdf.apply(lambda v: int(re.findall('/6\d/', v)[0][1:-1])))

In [10]:
pages = list(_bb.groupby(['pdf', 'pagenum']).groups.items())

In [11]:
interested_doc = np.random.choice(
    _bb.image_path,
    1000)
# interested_doc = ['pdf/62/A20181002170030.pdf']

In [12]:
# len(set([idx for x in interested_doc for idx in pages[x][1]]))
interested_index = _bb.image_path.isin(interested_doc)

In [13]:
# interested_index = [idx for x in interested_doc for idx in pages[x][1]]
bb=_bb.copy()[interested_index]
bb

,pdf,pagenum,image_path,x0,y0,x1,y1,text,ocr-text,fisical_year
1818,pdf/61/A20171003165836.pdf,19,A20171003165836/19.png,885.390015,138.894505,943.602173,181.014496,180,NaN,61
1819,pdf/61/A20171003165836.pdf,19,A20171003165836/19.png,255.311989,286.554405,294.084274,328.674408,05,NaN,61
1820,pdf/61/A20171003165836.pdf,19,A20171003165836/19.png,527.099991,286.554405,553.975021,328.674408,5.,NaN,61
1821,pdf/61/A20171003165836.pdf,19,A20171003165836/19.png,563.407425,286.554405,699.662476,328.674408,ค่าครุภัณฑ์,NaN,61
1822,pdf/61/A20171003165836.pdf,19,A20171003165836/19.png,708.990005,286.554405,953.372314,328.674408,ที่ดินและสิ่งก่อสร้าง,NaN,61
...,...,...,...,...,...,...,...,...,...,...
1692013,pdf/65/65071.pdf,41,65071/41.png,1276.050018,2142.497864,1401.399445,2188.829956,"1,100,000",NaN,65
1692014,pdf/65/65071.pdf,41,65071/41.png,1488.899963,2142.497864,1539.780945,2188.829956,บาท,NaN,65
1692015,pdf/65/65071.pdf,41,65071/41.png,352.500000,2198.717834,452.317520,2245.049927,07199-1,NaN,65
1692016,pdf/65/65071.pdf,41,65071/41.png,512.339996,2198.717834,803.220703,2245.049927,ค่ำใช้จ่ำยในกำรบ/ำรุงรักษำ,NaN,65


fix text by dict

In [14]:
chr_fix_pair={'ำ': 'า','/า': 'ำ', '/้า': '้ำ', '/่':'่'}

In [15]:
def fix_pdf_text(old_text):
    new_text = old_text.strip()
    for o,n in chr_fix_pair.items():
        new_text=new_text.replace(o,n)
    return new_text

bb=bb.assign(fix_text=bb.text.apply(fix_pdf_text))

replace text from pdf with OCR text

In [16]:
bb.loc[~bb['ocr-text'].isna(), 'fix_text'] = bb[~bb['ocr-text'].isna()]['ocr-text']

group line

In [17]:
for g, page_df in tqdm(bb.groupby('image_path')):
    if len(page_df) > 1000:
        print('skiped',g)
        continue
    page_df = page_df.sort_values(['y0', 'x0'])
    bb.loc[page_df.index, 'line_label']=(
        (page_df.y0.diff() > 10) | page_df.y0.diff().isna()
    ).astype(int).cumsum()
bb = bb[~bb.line_label.isna()]

  0%|          | 0/763 [00:00<?, ?it/s]

skiped A20171003161804/08.png
skiped A20171003161804/13.png


In [233]:
bb.loc[:, 'line_label'] = bb.line_label.astype(int)

/Users/napatswift/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [234]:
# bb.sort_values(['fisical_year', 'pagenum', 'line_label', 'y0', 'x0'])

# Try to parse

In [235]:
import fitz

In [244]:
get_patern_of_bullet('05-32')

('^\\d[\\-\\d]+$', 70)

In [243]:
import re
def get_patern_of_bullet(String):
    regx = [
        (r'^\d[\-\d]+$', 70),
        (r'^\(\d*(\.?\d*)*\)$', 50),
        (r'^[1-9]\d*(\.[1-9]\d*)*\)$', 20),
        (r'^[1-9]\d*(\.[1-9]\d*)+$', 5),
        (r'^[1-9]\d*\.$', 4),
        (r'^โครงการ', 3),
        (r'^งาน', 3),
        (r'^แผนงาน', 2),
        (r'^[\u0E00-\u0E7F]\.$', 1)
    ]
    
    if String in ['ด้านการจัดบริการของสำนักงานเขต',
                  'ด้านการบริหารจัดการและบริหารราชการกรุงเทพมหานคร',
                  'ด้านการศึกษา',
                  'ด้านความปลอดภัยและความเป็นระเบียบเรียบร้อย',
                  'ด้านทรัพยากรธรรมชาติและสิ่งแวดล้อม',
                  'ด้านพัฒนาสังคมและชุมชนเมือง',
                  'ด้านการสาธารณสุข',
                  'ด้านสาธารณสุข',
                  'ด้านเมืองและการพัฒนาเมือง',
                  'ด้านการระบายนำและบำบัดนำเสีย',
                  'ด้านการบริหารทั่วไป',
                  'ด้านบริหารทั่วไป',
                  'ด้านเศรษฐกิจและการพาณิชย์']: return ('ด้าน',1)

    for r, l in regx:
        if re.match(r, String):
            if l in [5, 20, 50]:
                l = String.count('.') + l
            if r == '^งาน' and String == 'งานที่จะทำ': continue
            return r, l
    return '', 0

In [237]:
def get_char_type(char):
    char_types = ['[\d\-]', '[\u0E00-\u0E7F]', '[a-zA-Z]']
    
    for i, ctype in enumerate(char_types):
        if re.match(ctype, char) is not None:
            return i
    return -1

def split_text(text):
    text = ['[START]'] + list(text)
    splits = list()
    token = list()
    for ci in range(len(text)-1) :
        c0, c1 = text[ci], text[ci+1]
        if get_char_type(c0) == get_char_type(c1) or c1 in [' ', '.', ','] or c0 in [',', '.'] or c0 == '[START]':
            pass
        else:
            splits.append(''.join(token))
            token = list()
        if c1 != ' ':
            token.append(c1)
    splits.append(''.join(token))
    return splits

In [238]:
budget_resource = json.load(open('budget_resource.json'))
document_index = {
    pdf['path']: re.sub('^[^\s]* ', '',pdf['name'])
    for year in budget_resource
    for pdf in budget_resource[year]['pdf_path']
}
document_url_index = {
    pdf['path']: pdf['url']
    for year in budget_resource
    for pdf in budget_resource[year]['pdf_path']
}

In [249]:
bb_with_entry = bb.copy()

In [250]:
entry_label_id = 0
is_entry = False
prog_bar = tqdm(total=len(bb[~bb.line_label.isna()].groupby(['pdf', 'pagenum'])))

for pdf_name, pdf_df in bb[~bb.line_label.isna()].groupby(['pdf',]): 
    doc = fitz.open(pdf_name)
#     if pdf_name in except_pdf:
#         prog_bar.update(len(pdf_df))
#         continue
    
    for pdf_page_index, page_df in pdf_df.groupby(['pagenum']):
        prog_bar.update(1)

        page = doc.load_page(pdf_page_index)

        if [x for x in page.get_drawings() if x['rect'].height > 10]:
#             prog_bar.update(len(pdf_df[pdf_df.pagenum > pdf_page_index].groupby(['pagenum'])))
            continue

        bb_with_entry.loc[page_df.index, 'is_included'] = True
            
        for line_num, line_df in page_df.groupby('line_label'):
            if (line_df.fix_text == 'รายละเอียดรายจ่าย').any():
                continue
            
#             fisical_year = line_df.fisical_year.values[0]
#             if (line_df.fix_text.isin(['ปี', f'25{fisical_year}', 'บาท'])).sum() == 3:
#                 continue
            if line_num < 2 or ''.join(line_df.fix_text.values).isdigit():
                continue

            text_line = line_df.sort_values(by='x0',ascending=True).fix_text.values
            text_line = split_text(' '.join(text_line))
            bullet_code = get_patern_of_bullet(text_line[0])[1]
            
            if text_line == 'วัตถุประสงค์':
                bullet_code = 0
                is_entry = False

            if bullet_code == 70:
                if len(text_line) > 1:
                    second_bullet = get_patern_of_bullet(text_line[1])[1]
                    if second_bullet != 0: bullet_code = second_bullet
                if len(text_line) > 2:
                    third_bullet = get_patern_of_bullet(text_line[2])[1]
                    if third_bullet != 0: bullet_code = third_bullet

            if bullet_code != 0:
                entry_label_id += 1
                bb_with_entry.loc[line_df.index, 'bullet_label'] = bullet_code
                is_entry = True

            if is_entry:
                bb_with_entry.loc[line_df.index, 'entry_label'] = entry_label_id

            is_last_token_baht = text_line[-1] == 'บาท'
            if is_last_token_baht:
                is_entry = False

  0%|          | 0/788 [00:00<?, ?it/s]

In [251]:
# bb_with_entry[~bb_with_entry.is_included.isna()][[
#     'pdf','pagenum','line_label','fix_text','fisical_year','bullet_label','entry_label',]]\
# .to_csv('bb-with-entry.csv')

In [252]:
bb_with_entry[
    (bb_with_entry.pagenum == 24) & (bb_with_entry.pdf == 'pdf/64/topic2787_2020_10_06_10_18_51.pdf')
][['fix_text', 'entry_label']].values;

In [253]:
# inspect them
start_entry=bb_with_entry[~bb_with_entry.bullet_label.isna()].groupby(['pdf','pagenum','entry_label']).x0.min()
start_entry.mean(),start_entry.std(),start_entry.mean()+start_entry.std(),start_entry.max()

(279.8623687805218, 120.39865005477112, 400.26101883529293, 1516.980010986328)

In [254]:
len(bb_with_entry.pdf.unique())

295

In [256]:
bb_with_entry\
    .groupby(['pdf','pagenum','line_label'])\
    .apply(lambda x: pd.Series({'text': ' '.join(x.sort_values(['x0']).fix_text.values),
                                'entry': x.entry_label.values[0]}))\
    .to_csv('dataset/rule_based_entry_by_line.csv')

In [198]:
bb_with_entry.to_csv('dataset/rule_based_entry.csv')

In [141]:
# test print by entry
bb_with_entry\
    .groupby(['pdf','pagenum', 'entry_label'])\
    .apply(lambda x: (
            x.iloc[0].pdf,
#             x.iloc[0].entry_label,
#             x.iloc[0].bullet_label,
            ' '.join(x.fix_text.values.tolist())
    )).to_list();

Relation extraction

pairing parent node with child nodes

In [ ]:
entry_bullet_table = bb_with_entry.groupby('entry_label').apply(lambda x: x.bullet_label.unique()[0])
entry_page_num_table = bb_with_entry.groupby('entry_label').apply(lambda x: x.pagenum.unique()[0])
entry_text_table = bb_with_entry.groupby('entry_label').apply(lambda x: ' '.join(x.sort_values('x0').fix_text.values))

In [ ]:
child_parent_table = dict()
curr = None
prev_pdf_name = None
for entry, bullet in entry_bullet_table.items():
    pdf_name = bb_with_entry[bb_with_entry.entry_label == entry].pdf.iloc[0]
    if prev_pdf_name != pdf_name:
        curr = None
    prev_pdf_name = pdf_name
    while (curr is not None and bullet <= entry_bullet_table[curr]):
        curr = child_parent_table[curr]
    child_parent_table[entry] = curr
    curr = entry

In [ ]:
len(child_parent_table),len(entry_bullet_table)

# Build Graph

In [ ]:
from graphviz import Digraph

In [ ]:
d = Digraph('diagram', graph_attr={'rankdir': 'RL'})

pdf_in_diagram = []

d.attr('node',shape='circle')
print('total pdf', len(bb_with_entry.pdf.unique()))
for pdf_name in bb_with_entry.pdf.unique():
    d.node(pdf_name)

d.attr('node',shape='box')
for c,p in child_parent_table.items():
    cs = str(c)
    ps = str(p)
    
    pdf_name = bb_with_entry[bb_with_entry.entry_label == c].pdf.iloc[0]
    node_label = entry_text_table.loc[c]
    
    amount = re.findall('[\d,]+ บาท', node_label)
    node_label = re.sub('[\d,]+ บาท', '', node_label)
    node_label = node_label[:50].strip() + ' ' + (amount[0] if len(amount) > 0 else '')
    
    d.node(cs, node_label)
    
    if p is None:
        ps = pdf_name
        if pdf_name not in pdf_in_diagram:
            pdf_in_diagram.append(pdf_name)

    d.edge(cs, ps)
    pagenum = entry_page_num_table[c]

In [ ]:
[split_text(t) for t in entry_text_table.sample(10).values]

In [ ]:
d.render('out',format='svg')

In [ ]:
bb_with_entry[bb_with_entry.pdf == 'pdf/62/A20181002170030.pdf']\
.groupby(['pagenum', 'line_label'])\
.apply(lambda x: print(x[['bullet_label', 'fix_text']].values.tolist()));

# Contruct table

In [ ]:
doc_count = {}
for doc_name in document_index.values():
    doc_name = re.sub('\s','',doc_name)
    doc_count[doc_name] = doc_count.get(doc_name, 0) + 1
doc_names = [k for k, v in doc_count.items() if v > 3]

## Get department name

In [ ]:
doc_org_name = {}
for pdf, pdf_df in bb_with_entry[(bb_with_entry.pagenum == 0)].groupby(['pdf',]):
    for l, line_df in pdf_df.groupby(['line_label']):
        line_text = line_df.fix_text.values.tolist()[0]
        if re.findall('[\u0E00-\u0E7F]+', line_text):
            print(pdf, line_text)
            doc_org_name[pdf] = line_text
            break

## structuring

In [ ]:
rows = []
for c, p in child_parent_table.items():
    is_leaf = c not in child_parent_table.values()
    if not is_leaf: continue

    curr_p = p
    ancesters = []
    while curr_p is not None:
        ancesters.append(curr_p)
        curr_p = child_parent_table[curr_p]

    ancesters = ancesters[::-1]
    
    pdf_name = bb_with_entry[bb_with_entry.entry_label == c].pdf.iloc[0]
    org = doc_org_name[pdf_name]
    ancesters.append(c)
    entries = [entry_text_table.loc[a] for a in ancesters]

    temp_row = dict()
    for i, entry in enumerate(entries):
        
        entry = entry.replace('- 6', '-6')
        amount = re.findall('([\d,]+) ?บาท', entry)
        entry = re.sub('[\d,]+ ?บาท', '', entry)
        bullet = re.findall('^[\d\.\-–]+',entry)
        entry = re.sub('^[\d\.\-–]+', '',entry).strip()
        entry = re.sub('^([\d\.]+|[\u0E00-\u0E7F]\.) ?', '',entry)
#         entry = re.sub('[\d\u0E00-\u0E7F\.]+', '',entry)
        entry = entry.strip()
        
#         if not amount:
#             print(entry)
#         if not bullet:
#             print(entry_text_table.loc[ancesters[i]])
        if i == len(entries) - 1:
            if bullet:
                temp_row['output/proj'] = bullet[0]
            temp_row['output_proj_name'] = entry
            if amount:
                temp_row['amount'] = amount[0]
        else:
            if bullet:
                temp_row[f'bullet_{i}'] = bullet[0]
            temp_row[f'name_{i}'] = entry
            if amount:
                temp_row[f'amount_{i}'] = amount[0]

    row = {'name_organization': org, 'pdf_name': pdf_name}
    row.update(temp_row)
    rows.append(row)

In [92]:
f'total of: {len(rows)}'

'total of: 1578'

In [93]:
bkk_budget = pd.DataFrame(rows)
bkk_budget.sample(5)

,name_organization,pdf_name,bullet_0,name_0,amount_0,output/proj,output_proj_name,amount,name_1,amount_1,bullet_1,bullet_2,name_2,amount_2
1379,สำนักงานเขตตลิ่งชัน,pdf/63/A20190917182422.pdf,07,ด้านการศึกษา,"85,567,600",0725094,งานบริหารการศึกษา,"85,567,600",แผนงานบริหารการศึกษา,"85,567,600",0725,NaN,NaN,NaN
196,สารบัญ,pdf/61/A20171003161852.pdf,NaN,แผนงานบริหารการคลัง,"2,819,616,700",8.,เพื่อการปฏิบัติงานของกองบำเหน็จบำนาญ โครงการจั...,"4,119,600",NaN,NaN,NaN,NaN,NaN,NaN
1488,สำนักงานเขตจตุจักร,pdf/64/topic2787_2020_10_06_10_17_37.pdf,01,ด้านการบริหารทั่วไป,"47,146,310",0103006,งานบริหารการจัดเก็บรายได้,"10,795,910",แผนงานบริหารการคลัง,"15,963,030",0103,NaN,NaN,NaN
610,สารบัญ,pdf/61/A20171003161852.pdf,NaN,แผนงานรักษาความสะอาด,"232,619,200",NaN,งานรักษาความสะอาด,"232,619,200",NaN,NaN,NaN,NaN,NaN,NaN
851,สารบัญ,pdf/61/A20171003161852.pdf,NaN,แผนงานบริหารงานปกครองและทะเบียน,"28,266,140",2.,งานทะเบียน,"6,639,200",NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
bkk_budget = bkk_budget.assign(pdf_link=bkk_budget.pdf_name.apply(lambda name: document_url_index[name[4:]]))

In [95]:
bkk_budget_proj = bkk_budget[bkk_budget['output/proj'].apply(lambda bullet: len(str(bullet)) > 7)]
bkk_budget_proj

,name_organization,pdf_name,bullet_0,name_0,amount_0,output/proj,output_proj_name,amount,name_1,amount_1,bullet_1,bullet_2,name_2,amount_2,pdf_link
1099,สำนักการคลัง,pdf/62/A20181002153225.pdf,01,ด้านการบริหารทั่วไป,"3,754,118,000",0103016-60-01,โครงการจัดหาระบบเทคโนโลยีสารสนเทศ เพื่อการปฏิบ...,"5,160,200",แผนงานบริหารการคลัง,"3,476,296,600",0103,NaN,NaN,NaN,http://office.bangkok.go.th/budd/main/upload/2...
1102,สำนักการคลัง,pdf/62/A20181002153225.pdf,01,ด้านการบริหารทั่วไป,"3,754,118,000",0105024-62-01,โครงการพัฒนาระบบสารสนเทศกองโรงงานช่างกล,"12,099,600",แผนงานส่งเสริมระบบบริหาร,"255,299,300",0105,NaN,NaN,NaN,http://office.bangkok.go.th/budd/main/upload/2...
1124,สำนักงานเขตบางขุนเทียน,pdf/62/A20181002165136.pdf,03,ด้านการโยธาและระบบจราจร,"102,339,700",0310037-62-10,โครงการก่อสร้างสะพานทางเดิน,"7,497,600",แผนงานการโยธา,"102,339,700",0310,NaN,NaN,NaN,http://office.bangkok.go.th/budd/main/upload/2...
1185,สำนักงานเขตดินแดง,pdf/62/A20181002165913.pdf,03,ด้านการโยธาและระบบจราจร,"96,341,200",0310037-62-13,โครงการก่อสร้างสำนักงานเขตดินแดงแห่งใหม่,"75,000,000",แผนงานการโยธา,"96,341,200",0310,NaN,NaN,NaN,http://office.bangkok.go.th/budd/main/upload/2...
1222,สำนักงานเขตมีนบุรี,pdf/62/A20181004120240.pdf,03,ด้านการโยธาและระบบจราจร,"92,310,300",0310037-62-12,โครงการปรับปรุงซอยบึงขวาง เชิงสะพานข้ามคลองศาล...,"21,167,700",แผนงานการโยธา,"92,310,300",0310,NaN,NaN,NaN,http://office.bangkok.go.th/budd/main/upload/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,สำนักงานเขตยานนาวา,pdf/64/topic2787_2020_10_06_10_16_39.pdf,04,ด้านการระบายน้ำและบำบัดน้ำเสีย,"17,449,800",0413045-63-08,โครงการก่อสร้างเขื่อน คลองโบสถ์ จากเขื่อนเดิมถ...,"7,967,100",แผนงานจัดการระบายน้ำและแก้ไขปัญหาน้ำท่วม,"17,449,800",0413,NaN,NaN,NaN,http://office.bangkok.go.th/budd/main/upload/2...
1495,สำนักงานเขตจตุจักร,pdf/64/topic2787_2020_10_06_10_17_37.pdf,05,ด้านการพัฒนาและบริการสังคม,"63,050,200",0517057-63-01,โครงการก่อสร้างอาคารศูนย์ฝึกอาชีพ สำนักงานเขตจ...,"6,800,200",แผนงานพัฒนาชุมชน,"33,274,400",0517,NaN,NaN,NaN,http://office.bangkok.go.th/budd/main/upload/2...
1510,สำนักงานเขตบึงกุ่ม,pdf/64/topic2787_2020_10_06_10_17_46.pdf,07,ด้านการศึกษา,"104,277,900",0725094-63-05,โครงการก่อสร้างอาคารบ้านพักครู ค.ส.ล. 4 ชั้น,"9,068,900",แผนงานบริหารการศึกษา,"104,277,900",0725,NaN,NaN,NaN,http://office.bangkok.go.th/budd/main/upload/2...
1540,สำนักวัฒนธรรม,pdf/64/topic2787_2020_10_08_10_32_05.pdf,05,ด้านการพัฒนาและบริการสังคม,"1,323,686,200",0516065-63-02,โครงการก่อสร้างศูนย์เยาวชนวัดดอกไม้,"3,300,000",แผนงานบริการสังคม,"671,483,900",0516,NaN,NaN,NaN,http://office.bangkok.go.th/budd/main/upload/2...


In [96]:
# bkk_budget_proj.to_csv('bkkbudget_61-64_v0.csv')